In [1]:
import numpy as np
import pandas as pd

In [2]:
trained_df = pd.read_csv("trained_strategy.csv")
trained_df.head(5)

,player_sum,dealer_visible,usable_ace,can_split,can_double,best_action,ev
0,2,2,False,False,True,Hit,-2.0000
1,2,2,False,False,False,Hit,-0.3482
2,2,3,False,False,True,Stand,0.0000
3,2,3,False,False,False,Stand,-0.5000
4,2,4,False,False,True,Stand,2.0000


In [3]:
basic_df = pd.read_csv("basic_strategy.csv")
basic_df.head(5)

,player_sum,dealer_visible,usable_ace,can_split,can_double,best_action,ev
0,2,4,False,False,True,Hit,-0.3137
1,2,4,False,False,False,Hit,-0.0396
2,2,5,False,False,True,Hit,-0.2442
3,2,5,False,False,False,Hit,0.0384
4,2,6,False,False,True,Hit,0.1599


In [4]:
df = pd.merge(
    trained_df,
    basic_df,
    on=['player_sum', 'dealer_visible', 'usable_ace', 'can_split', 'can_double'],
    how='outer',
    suffixes=('_trained', '_basic')
)

In [5]:
if df.duplicated(subset=['player_sum', 'dealer_visible', 'usable_ace', 'can_split', 'can_double']).any():
    print("Duplicated rows found.")
else:
    print("No duplicate rows found.")

No duplicate rows found.


In [6]:
print(f"There are {df.isna().any(axis=1).sum()} rows with missing action data.")

There are 64 rows with missing action data.


In [7]:
def choose_best_action_with_source(row):
    trained = row['ev_trained']
    basic = row['ev_basic']

    if pd.notna(trained) and pd.isna(basic):
        return row['best_action_trained'], trained, 'trained'
    elif pd.notna(basic) and pd.isna(trained):
        return row['best_action_basic'], basic, 'basic'
    elif pd.notna(trained) and pd.notna(basic):
        if trained >= basic:
            return row['best_action_trained'], trained, 'trained'
        else:
            return row['best_action_basic'], basic, 'basic'
    else:
        return np.nan, np.nan, np.nan

# Apply and split into three columns
df[['best_action', 'ev', 'source']] = df.apply(
    choose_best_action_with_source, axis=1, result_type='expand'
)

In [9]:
df_cleaned = df.dropna()

print(f"There are {df_cleaned.isna().any(axis=1).sum()} rows with missing action data after imputation.")

There are 0 rows with missing action data after imputation.


In [24]:
uniq_source_counts = df_cleaned['source'].value_counts()

print(f"Among all finalized expected values, {uniq_source_counts.iloc[0]} are sourced from basic strategy\
 results and {uniq_source_counts.iloc[1]} from trained model results.")

Among all finalized expected values, 413 are sourced from basic strategy results and 226 from trained model results.
